In [58]:
import numpy as np
import functools 
import math
import torch
import sklearn
import sys
sys.path.insert(1,"./src")

from NNCPD import NNCPD, weights_H, Energy_Loss_Tensor2, Energy_Loss_Tensor, Recon_Loss, L21_Norm, outer_product, outer_product_np, PTF, random_NNCPD, Fro_Norm



import torch
import numpy as np
from matplotlib import pyplot as plt
import sys
sys.path.insert(1,"./src")

from NNCPD import NNCPD, weights_H, Energy_Loss_Tensor2, Energy_Loss_Tensor, Recon_Loss, L21_Norm, outer_product, outer_product_np, PTF, random_NNCPD, Fro_Norm
from lsqnonneg_module import LsqNonneg
from trainNNCPD import train
#
import torch.nn as nn
from torch.autograd import Variable
from writer import Writer


import tensorly as tl
from tensorly import unfold as tl_unfold
from tensorly.decomposition import parafac, non_negative_parafac

torch.set_default_tensor_type(torch.DoubleTensor)

In [54]:
# Function to enerate data
def hnmfdatagen(cs, std=0):
    if len(cs) > 2:
        matrices = [np.random.rand(cs[i], cs[i+1]) for i in range(len(cs)-1)]
        return functools.reduce(np.matmul, matrices[1:], matrices[0]) + np.random.normal(0, std, [cs[0],cs[-1]])
        # we might want to scale our generated data

In [55]:
# Zero out proportion of generated entries
def zero_entries(X, p_miss, mecha="MCAR", opt=None, p_obs=None, q=None):
    """
    Generate missing values for specifics missing-data mechanism and proportion of missing values. 
    
    Parameters
    ----------
    X : torch.DoubleTensor or np.ndarray, shape (n, d)
        Data for which missing values will be simulated.
        If a numpy array is provided, it will be converted to a pytorch tensor.
    p_miss : float
        Proportion of missing values to generate for variables which will have missing values.
    mecha : str, 
            Indicates the missing-data mechanism to be used. "MCAR" by default, "MAR", "MNAR" or "MNARsmask"
    opt: str, 
         For mecha = "MNAR", it indicates how the missing-data mechanism is generated: using a logistic regression ("logistic"), quantile censorship ("quantile") or logistic regression for generating a self-masked MNAR mechanism ("selfmasked").
    p_obs : float
            If mecha = "MAR", or mecha = "MNAR" with opt = "logistic" or "quanti", proportion of variables with *no* missing values that will be used for the logistic masking model.
    q : float
        If mecha = "MNAR" and opt = "quanti", quantile level at which the cuts should occur.
    
    Returns
    ----------
    A dictionnary containing:
    'X_init': the initial data matrix.
    'X_incomp': the data with the generated missing values.
    'mask': a matrix indexing the generated missing values.s
    """
    
    to_torch = torch.is_tensor(X) ## output a pytorch tensor, or a numpy array
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)
    
    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()
    
    X_nas = X.clone()
    X_nas[mask.bool()] = 0
    
    return {'X_init': X.double(), 'X_incomp': X_nas.double(), 'mask': mask}

In [56]:
X = hnmfdatagen([16, 8, 4, 16])
X = zero_entries(X, p_miss=0.33333333, mecha="MCAR")
display(dict.keys(X))

dict_keys(['X_init', 'X_incomp', 'mask'])

In [59]:
r=8
n1,n2 = X['X_incomp'].shape
net = NNCPD([n1,4],[n2,4],[0,0])

loss_func = Energy_Loss_Tensor()

history_unsupervised = train(net, X['X_incomp'], loss_func, r, epoch = 15000, lr1 = 0, lr2 = 0.1, random_init=True)


TypeError: expected np.ndarray (got list)

<function Tensor.type>